<a href="https://colab.research.google.com/github/awal015/CapstoneStep4/blob/main/BoardGameRecommenderPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'board-games-database-from-boardgamegeek:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1870426%2F3055483%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241115%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241115T211044Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D894464b0743907a1518e3a11808f8056fb808bfb899762045fbfd432f050f35f926228fc2e098f0b108e662db9dbf512dacf512731b04476bd631df4e3aa43cb4d8e3e716bee10683b5eca94576cc9df538f466db284ce8c476e4d14107240db1cd8ef085d884b031004bb3f743d3c04307ed1a3866f195f36d20d59cf413b77e90b5c1a92261c9322d067232e4c20ef53c465d0ae2f2a42cd7f9a101681c545f72efdc22f94b90f8664e0eee6ce593a40c7f9faca929c5c7b58f78450429eee6ea67142d8839cf28ea5c4dea2ec1fd8453a4ff84f28756e713008bbe45df11986c5e5e37a8fa592b1c323a7547f8b41fd3bee564b96b7e89c482008f20ea813'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(ggplot2)
library(recommenderlab)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
user_ratings = read.csv('../input/board-games-database-from-boardgamegeek/user_ratings.csv')

games = read.csv('../input/board-games-database-from-boardgamegeek/games.csv')


In [ ]:
update = merge(user_ratings,games,by = 'BGGId')

In [ ]:
test_data = update[,c('Name','Rating','Username')]

In [ ]:
head(test_data)

In [ ]:
data = test_data[,c(1,3,2)]

In [ ]:
head(data)

In [ ]:
data_matrix = as(data,Class = 'realRatingMatrix')

In [ ]:
#colMeans(data_matrix)

In [ ]:
# image(data_matrix)

In [ ]:
str(data_matrix)

In [ ]:
set.seed(6969)
split = sample(x = nrow(data_matrix), size = 0.8*nrow(data_matrix))
train = data_matrix[split,]
test = data_matrix[-split,]

In [ ]:
summary(rowCounts(train))

In [ ]:
#normalized
#ggplot(data=data.frame(game_ratings = rowMeans(normalize(train))),aes(x=game_ratings))+
#  geom_histogram(fill='sienna')

In [ ]:
recom_popular = Recommender(train,
                            method='POPULAR',
                            parameter=list(normalize='center'))

In [ ]:
pred_popular_topN = predict(recom_popular,newdata=test,type='topNList',n=5)

In [ ]:
getList(pred_popular_topN)['Kartigan']

Practicing recommender systems, working on getting past the prediction portion. Prediction model drains resources and forces a reboot. May be more practical in python.